In [1]:
logdir = 'data_array/'
import numpy as np
x = np.random.random((2000, 2000))

In [18]:
import pickle
%time pickle.dump(x,open(logdir+'x.pkl','wb'),protocol=0)

CPU times: user 46.3 ms, sys: 56.4 ms, total: 103 ms
Wall time: 102 ms


In [6]:
import _pickle as cPickle
%time cPickle.dump(x,open(logdir+'x.cpkl','wb'))

In [10]:
import hickle
%time hickle.dump(x,open(logdir+'x.hkl','w'), compression='gzip')

CPU times: user 930 ms, sys: 19.8 ms, total: 950 ms
Wall time: 948 ms


In [5]:
import hickle
k = hickle.load(logdir+'x.hkl')

In [6]:
k

array([[0.94395819, 0.92556078, 0.13164182, ..., 0.77651764, 0.41997287,
        0.95955858],
       [0.61121819, 0.97678407, 0.65052025, ..., 0.06337379, 0.22213284,
        0.44793331],
       [0.01702354, 0.04558505, 0.2679173 , ..., 0.08493753, 0.437424  ,
        0.04649727],
       ...,
       [0.32789776, 0.65952641, 0.50600896, ..., 0.82465462, 0.90750269,
        0.71215088],
       [0.72635393, 0.60582504, 0.66100743, ..., 0.14946469, 0.86971202,
        0.18332081],
       [0.76351665, 0.0409705 , 0.35853323, ..., 0.21948575, 0.31904432,
        0.59822802]])

In [14]:
from json_tricks import dump, dumps, load, loads, strip_comments
%time dump(x,open(logdir+'x.json','w'))

In [11]:
import h5py
with h5py.File(logdir+'x.h5_gzip', 'w') as f:
    def write(dsetname, a):
        f.create_dataset(dsetname, data=a, compression='gzip')
    %time write('x',x)

CPU times: user 982 ms, sys: 15.8 ms, total: 997 ms
Wall time: 996 ms


In [9]:
import h5py
with h5py.File(logdir+'x.h5_lzf', 'w') as f:
    def write(dsetname, a):
        f.create_dataset(dsetname, data=a, compression='lzf')
    %time write('x',x)

CPU times: user 156 ms, sys: 7.98 ms, total: 164 ms
Wall time: 165 ms


In [11]:
import bcolz
%time bcolz.carray(x, rootdir=logdir+'bcolz', mode='w')

CPU times: user 31.5 ms, sys: 26.5 ms, total: 58 ms
Wall time: 35.5 ms


carray((2000, 2000), float64)
  nbytes := 30.52 MB; cbytes := 27.14 MB; ratio: 1.12
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 32; chunksize: 512000; blocksize: 32768
  rootdir := 'data_array/bcolz'
  mode    := 'w'
[[ 0.44688809  0.19545426  0.2484356  ...,  0.21988661  0.97806542
   0.52447114]
 [ 0.68966819  0.41698948  0.56607713 ...,  0.49892793  0.28128249
   0.30648438]
 [ 0.72289576  0.27805977  0.87664277 ...,  0.18781709  0.55098415
   0.07410682]
 ..., 
 [ 0.97569222  0.34713028  0.31985369 ...,  0.4988987   0.04436345
   0.05717654]
 [ 0.08325323  0.08891162  0.59956499 ...,  0.58127371  0.51221409
   0.32769882]
 [ 0.43096533  0.41487155  0.17468498 ...,  0.26030677  0.35408746
   0.51229279]]

In [3]:
import array_pb2

In [5]:
def load_protobuf(path):
    pb_data = array_pb2.NDArray()
    pb_data.ParseFromString(open(path, 'r').read())
    return np.array(pb_data.array, dtype=pb_data.dtype).reshape(tuple(pb_data.shape))

In [6]:
parray = list(x.flatten().astype(int))
with open("test1.pbf", "wb") as pbf:
    arr = array_pb2.NDArray()
    arr.array.extend(parray)
    arr.shape.extend(list(x.shape))
    arr.dtype = x.dtype.name
    pbf.write(arr.SerializeToString())

# with open("test2.pbf", "wb") as pbf:
#     arr = array_pb2.NDArrayPacked()
#     arr.array.extend(parray)
#     arr.shape.extend(list(x.shape))
#     arr.dtype = x.dtype.name
#     pbf.write(arr.SerializeToString())

%time arr8 = load_protobuf("test1.pbf")
%time arr8 = load_protobuf("test2.pbf")

CPU times: user 569 ms, sys: 75.3 ms, total: 644 ms
Wall time: 644 ms
CPU times: user 490 ms, sys: 72.1 ms, total: 562 ms
Wall time: 562 ms
